# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 5

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task5`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [ ]:
!az login

## Step 1. Write the code

In [ ]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

In [ ]:
teamname="schrodingers-duck"  # Update this field with your team name
task=["task5"]
slack_id="U04L3SFUCQ2"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [ ]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task5_DumpMachineWrapper : qsharp.QSharpCallable = None
Task5_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 5 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [ ]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;


// Task 5. 
// (input will contain 6 qubits)
operation Task5(input : Qubit[], target : Qubit) : Unit is Adj {
    for i in [21,42] {
        ControlledOnInt(i, X)(input, target);
    }
    use equality = Qubit[3];
    use storageWire = Qubit[1];
    within {
    CNOT(input[0], equality[0]);
    CNOT(input[3], equality[0]);
    CNOT(input[1], equality[1]);
    CNOT(input[4], equality[1]);
    CNOT(input[2], equality[2]);
    CNOT(input[5], equality[2]);
    X(equality[0]); X(equality[1]); X(equality[2]);
    CCNOT(equality[0], equality[1], storageWire[0]);
    } apply {
    CCNOT(equality[2], storageWire[0], target);
    }
}

operation original(input : Qubit[], target : Qubit) : Unit is Adj {
    for i in [0, 9, 18, 21, 27, 36, 42, 45, 54, 63] {
        ControlledOnInt(i, X)(input, target);
    }
}

operation QubitArrayWrapperOperation (op : ((Qubit[], Qubit) => Unit is Adj), qs : Qubit[]) : Unit is Adj {        
    op(Most(qs), Tail(qs));
}

operation TestCompoundGate () : Unit {
    AssertOperationsEqualReferenced(6+1, QubitArrayWrapperOperation(Task5, _), QubitArrayWrapperOperation (original, _));
}

In [6]:
TestCompoundGate();

Source,Callable
(notebook),Task5
(notebook),QubitArrayWrapperOperation
/home/jovyan/2023_microsoft/D:\a\1\s\submodules\qsharp-runtime\src\Simulation\TargetDefinitions\Decompositions\AssertOperationsEqualReferenced.qs:0,Microsoft.Quantum.Diagnostics.AssertOperationsEqualReferenced
(notebook),TestCompoundGate


Specified argument was out of the range of valid values.


In [5]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task5_DumpMachineWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task5(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task5_ResourceEstimationWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    Task5(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [6]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
qsharp.config["dump.truncateSmallAmplitudes"]=True
Task5_DumpMachineWrapper.simulate()

|0000001⟩	0.12500000000000008 + 0𝑖
|0000010⟩	0.12500000000000008 + 0𝑖
|0000100⟩	0.12500000000000008 + 0𝑖
|0000110⟩	0.12500000000000008 + 0𝑖
|0001000⟩	0.12500000000000008 + 0𝑖
|0001010⟩	0.12500000000000008 + 0𝑖
|0001100⟩	0.12500000000000008 + 0𝑖
|0001110⟩	0.12500000000000008 + 0𝑖
|0010000⟩	0.12500000000000008 + 0𝑖
|0010011⟩	0.12500000000000008 + 0𝑖
|0010100⟩	0.12500000000000008 + 0𝑖
|0010110⟩	0.12500000000000008 + 0𝑖
|0011000⟩	0.12500000000000008 + 0𝑖
|0011010⟩	0.12500000000000008 + 0𝑖
|0011100⟩	0.12500000000000008 + 0𝑖
|0011110⟩	0.12500000000000008 + 0𝑖
|0100000⟩	0.12500000000000008 + 0𝑖
|0100010⟩	0.12500000000000008 + 0𝑖
|0100101⟩	0.12500000000000008 + 0𝑖
|0100110⟩	0.12500000000000008 + 0𝑖
|0101000⟩	0.12500000000000008 + 0𝑖
|0101011⟩	0.12500000000000008 + 0𝑖
|0101100⟩	0.12500000000000008 + 0𝑖
|0101110⟩	0.12500000000000008 + 0𝑖
|0110000⟩	0.12500000000000008 + 0𝑖
|0110010⟩	0.12500000000000008 + 0𝑖
|0110100⟩	0.12500000000000008 + 0𝑖
|0110111⟩	0.12500000000000008 + 0𝑖
|0111000⟩	0.12500000000000008 + 0𝑖
|0111010⟩	0.12500000000000008 + 0𝑖
|0111100⟩	0.12500000000000008 + 0𝑖
|0111110⟩	0.12500000000000008 + 0𝑖
|1000000⟩	0.12500000000000008 + 0𝑖
|1000010⟩	0.12500000000000008 + 0𝑖
|1000100⟩	0.12500000000000008 + 0𝑖
|1000110⟩	0.12500000000000008 + 0𝑖
|1001001⟩	0.12500000000000008 + 0𝑖
|1001010⟩	0.12500000000000008 + 0𝑖
|1001100⟩	0.12500000000000008 + 0𝑖
|1001110⟩	0.12500000000000008 + 0𝑖
|1010000⟩	0.12500000000000008 + 0𝑖
|1010010⟩	0.12500000000000008 + 0𝑖
|1010101⟩	0.12500000000000008 + 0𝑖
|1010110⟩	0.12500000000000008 + 0𝑖
|1011000⟩	0.12500000000000008 + 0𝑖
|1011011⟩	0.12500000000000008 + 0𝑖
|1011100⟩	0.12500000000000008 + 0𝑖
|1011110⟩	0.12500000000000008 + 0𝑖
|1100000⟩	0.12500000000000008 + 0𝑖
|1100010⟩	0.12500000000000008 + 0𝑖
|1100100⟩	0.12500000000000008 + 0𝑖
|1100110⟩	0.12500000000000008 + 0𝑖
|1101000⟩	0.12500000000000008 + 0𝑖
|1101010⟩	0.12500000000000008 + 0𝑖
|1101101⟩	0.12500000000000008 + 0𝑖
|1101110⟩	0.12500000000000008 + 0𝑖
|1110000⟩	0.12500000000000008 + 0𝑖
|1110010⟩	0.12500000000000008 + 0𝑖
|1110100⟩	0.12500000000000008 + 0𝑖
|1110110⟩	0.12500000000000008 + 0𝑖
|1111000⟩	0.12500000000000008 + 0𝑖
|1111010⟩	0.12500000000000008 + 0𝑖
|1111100⟩	0.12500000000000008 + 0𝑖
|1111111⟩	0.12500000000000008 + 0𝑖

()

## Step 3. Evaluate the code using resource estimation

In [8]:
# If you're using this notebook in Azure Quantum hosted notebooks, remove the credential="CLI" parameter!
# If you're using this notebook in qBraid, keep it
qsharp.azure.connect(
    resourceId="/subscriptions/7fc54e95-14cf-4a6f-b50e-d6155c241309/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/azure-q",
    location="UK South",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace azure-q in location uksouth.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 284985},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 412930},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 2},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 257411},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 4298},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 137},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current

In [9]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [10]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task5_ResourceEstimationWrapper, jobName="RE for the task 5")

Submitting Task5_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 5
   Job ID: ce9dd9e8-528c-46ec-94d2-289b8fed02a7
Waiting up to 30 seconds for Azure Quantum job to complete...
[11:30:22 PM] Current job status: Executing
[11:30:27 PM] Current job status: Succeeded


In [11]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,102546,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 5346 physical qubits to implement the algorithm logic, and 97200 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,169us 200ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (3us 600ns) multiplied by the 47 logical cycles to run the algorithm. If however the duration of a single T factory (here: 46us 800ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,33,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 11\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 33$ logical qubits."
Algorithmic depth,47,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 8 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 5 CCZ and 8 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,47,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 47. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,52,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 5 CCZ and 8 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,15,Number of T factories capable of producing the demanded 52 T states during the algorithm's runtime The total number of T factories 15 that are executed in parallel is computed as \(\left\lceil\dfrac{52\;\text{T states} \cdot 46us 800ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 169us 200ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,4,"Number of times all T factories are invoked In order to prepare the 52 T states, the 15 copies of the T factory are repeatedly invoked 4 times."
Physical algorithmic qubits,5346,Number of physical qubits for the algorithm after layout The 5346 are the product of the 33 logical qubits after layout and the 162 physical qubits that encode a single logical qubit.
Physical T factory qubits,97200,"Number of physical qubits for the T factories Each T factory requires 6480 physical qubits and we run 15 in parallel, therefore we need $97200 = 6480 \cdot 15$ qubits."
Required logical qubit error rate,3.22e-7,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 33 logical qubits and the total cycle count 47.


In [12]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [13]:
evaluate_results(result)

Logical algorithmic qubits = 33
Algorithmic depth = 47
Score = 1551


1551